In [23]:
import os

In [24]:
%pwd

'c:\\Users\\Pavithra\\Documents\\Notebooks\\Selfrojects\\Lung_Cancer_Classification_CT_Scan\\research'

In [30]:
os.chdir("../")

In [31]:
%pwd

'c:\\Users\\Pavithra\\Documents\\Notebooks\\Selfrojects\\Lung_Cancer_Classification_CT_Scan'

In [36]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [44]:
from src.lungClassifier.constants import *
from src.lungClassifier.utils.common import read_yaml, create_directories
from src.lungClassifier.entity.config_entity import DataIngestionConfig

In [45]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [48]:
import os
import zipfile
import gdown
from src.lungClassifier import logger
from src.lungClassifier.utils.common import get_size

In [49]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [43]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-07-29 13:05:01,330: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-29 13:05:01,351: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-29 13:05:01,355: INFO: common: created directory at: artifacts]
[2024-07-29 13:05:01,357: INFO: common: created directory at: artifacts/data_ingestion]
[2024-07-29 13:05:01,360: INFO: 2219526593: Downloading data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY&confirm=t&uuid=be638c85-aa30-4850-892a-d9464bc48db6
To: c:\Users\Pavithra\Documents\Notebooks\Selfrojects\Lung_Cancer_Classification_CT_Scan\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [00:04<00:00, 10.4MB/s]

[2024-07-29 13:05:11,848: INFO: 2219526593: Downloaded data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into file artifacts/data_ingestion/data.zip]
